In [1]:
# %pip install mediapipe opencv-python
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import math
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### Calculating Angle between two vectors

In [33]:
def calculate_angle(a, b, c):
    a = np.array([a.x, a.y]) # Start
    b = np.array([b.x, b.y]) # Mid
    c = np.array([c.x, c.y]) # End
    # print(a, b, c)
    
    radians = math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/math.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

### Exercise: Jumping Jacks

In [58]:
cap = cv2.VideoCapture(0)
# recording
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
record = cv2.VideoWriter('testing.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)
# initialize counter
counter = 0
stage = None
correct_pose = {'count': 0, 'match': False}
pose_count = 0
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Video ended")
            break
        # the BGR image to RGB.
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make detection
        results = pose.process(image)
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            # print(landmarks)
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]

            # calculating angle
            left_angle1 = calculate_angle(left_shoulder, left_elbow, left_wrist)
            left_angle2 = calculate_angle(left_hip, left_knee, left_ankle)
            right_angle1 = calculate_angle(right_shoulder, right_elbow, right_wrist)
            right_angle2 = calculate_angle(right_hip, right_knee, right_ankle)

            pose_count += 1
            if (left_angle1 > 150 and left_angle2 > 150) and (right_angle1 > 150 and right_angle2 > 150):
                correct_pose['count'] += 1
                correct_pose['match'] = True
            else:
                correct_pose['match'] = False
            
            angle1 = calculate_angle(left_elbow, left_shoulder, right_shoulder)
            angle2 = calculate_angle(right_elbow, right_shoulder, left_shoulder)
            angle3 = calculate_angle(right_hip, left_hip, left_knee)
            angle4 = calculate_angle(left_hip, right_hip, right_knee)
            print(angle1, angle2, angle3, angle4)
            cv2.putText(image, str(round(angle1,1)), tuple(np.multiply([left_shoulder.x, left_shoulder.y], [frame_width, frame_height]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(round(angle2,1)), tuple(np.multiply([right_shoulder.x, right_shoulder.y], [frame_width, frame_height]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(round(angle3,1)), tuple(np.multiply([left_hip.x, left_hip.y],[frame_width, frame_height]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(round(angle4,1)), tuple(np.multiply([right_hip.x, right_hip.y], [frame_width, frame_height]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            # count the reps
            if angle2 < 100 and angle4 < 90:
                stage = "down"
            if angle2 > 160 and angle4 > 100 and stage == 'down':
                stage = 'up'
                counter += 1
    
        except Exception as e:
            print(e)
        # render
        # status box    
        if(correct_pose['match']):
            color = (0, 255, 0)
        else:
            color = (0, 0, 255)
        cv2.rectangle(image, (0,0), (frame_width, 70), color, -1)
        # Reps
        cv2.putText(image, 'REPS', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        # Stage
        cv2.putText(image, 'STAGE', (100,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (80,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        # posture
        # cv2.putText(image, 'POSTURE', (175,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        # cv2.putText(image, "\U0001f600" if correct_pose['match'] else 'INCORRECT', (175,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        # print(stage)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                    )
                                    
        cv2.imshow('Raw Webcam Feed', image)
        record.write(image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
record.release()
cv2.destroyAllWindows()

100.24636481321721 107.24248291095549 90.42633973895806 88.23559130820335


QObject::moveToThread: Current thread (0x24b8820) is not the object's thread (0x2f4c790).
Cannot move to target thread (0x24b8820)

QObject::moveToThread: Current thread (0x24b8820) is not the object's thread (0x2f4c790).
Cannot move to target thread (0x24b8820)

QObject::moveToThread: Current thread (0x24b8820) is not the object's thread (0x2f4c790).
Cannot move to target thread (0x24b8820)

QObject::moveToThread: Current thread (0x24b8820) is not the object's thread (0x2f4c790).
Cannot move to target thread (0x24b8820)

QObject::moveToThread: Current thread (0x24b8820) is not the object's thread (0x2f4c790).
Cannot move to target thread (0x24b8820)

QObject::moveToThread: Current thread (0x24b8820) is not the object's thread (0x2f4c790).
Cannot move to target thread (0x24b8820)

QObject::moveToThread: Current thread (0x24b8820) is not the object's thread (0x2f4c790).
Cannot move to target thread (0x24b8820)

QObject::moveToThread: Current thread (0x24b8820) is not the object's thread

100.66958876863254 107.63733449050511 90.29430130905911 88.25141353074231
100.89415563639272 108.18268724288471 90.33945803394676 88.22257084588564
101.15420513556512 108.33107505510365 90.17206197109584 88.30283212439029
101.09390274178458 108.5073873797063 90.23496111593232 88.22687962823893
101.09108705825605 108.61371502756047 90.25985503358407 88.26099929703541
100.9661525451167 108.6108570185548 90.27569163290968 88.25493561591742
100.97258709062976 108.4789975006341 90.2874348983953 88.3031305498283
100.81011174898578 108.30732230376864 90.31283093950645 88.24691158384576
100.94782559797545 108.3047539629833 90.33508585191754 88.25458732258657
100.80073541116775 108.26513909055069 90.2595621045117 88.29717088884307
100.72215915274171 108.286371721839 90.21939461627454 88.32521136523904
100.48477003561294 108.39936939526083 90.02663323327494 88.42416550521243
100.28589182975492 108.31019443349567 89.85621807044896 88.57288137894136
100.19476835714124 108.21479433612723 89.7804119

In [51]:
print(f"total reps: {counter}")
print(f"total correct pose: {correct_pose['count']}")
print(f"total pose: {pose_count}")
print(f"accuracy: {round(correct_pose['count']/pose_count * 100, 2)}")

total reps: 2
total correct pose: 94
total pose: 134
accuracy: 70.15
